<a href="https://colab.research.google.com/github/mist-project/mist-v2/blob/main/mist_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Part 1: Mount the victin model**

Check computing resources by running the following command. For a free user, you should see outputs similar to the following, which means you have access to one T4 GPU.


<img src="https://drive.google.com/uc?export=view&id=1JaRIA6JrbkXOsE5rYpejCd3RqAdauYSA" alt="image" width="50%" height="auto">



In [1]:
!nvidia-smi

Thu Jan 18 04:24:35 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

Create a shortcut for the victim Stable Diffusion model. Right click on the *Diffuser* folder in our [Google Drive](https://drive.google.com/drive/folders/1vg8oK2BUOla5adaJcFYx5QMq0-MoP8kk) and choose *Organize* -> *Add shortcut*.

<img src="https://drive.google.com/uc?export=view&id=1oZ26ocm1KxK5ZmQ6GPQEHDQ2PCmtxGuH" alt="image" width="50%" height="auto">


Add a shortcut of the model to *MyDrive* directory.

<img src="https://drive.google.com/uc?export=view&id=1FoBq2zo785dSFIfG0tSrRUoHN2kUAEd8" alt="image" width="50%" height="auto">

Then, mount the the model to Colab.

In [2]:
# Mount data from drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


#**Part 2: Download Mist-v2**

In [3]:
!git clone https://github.com/mist-project/mist-v2.git
%cd mist-v2

Cloning into 'mist-v2'...
remote: Enumerating objects: 809, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 809 (delta 37), reused 29 (delta 13), pack-reused 725
Receiving objects: 100% (809/809), 31.30 MiB | 16.15 MiB/s, done.
Resolving deltas: 100% (356/356), done.
/content/mist-v2


In [5]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu121


Next, you need to create two folders to store images.

In [7]:
!mkdir /content/mist-v2/data/img
!mkdir /content/mist-v2/data/contrast

Then, download the surrogate model.

In [8]:
!cp -r /content/drive/MyDrive/Diffusers/stable-diffusion-1-5 /content/mist-v2/stable-diffusion

#**Part 3: Boot Mist-v2**


Before running Mist-v2, please upload your to-be-misted images to `/content/mist-v2/data/img/`. After that, running the following command.

In [ ]:
!accelerate launch attacks/mist.py \
--pgd_eps 0.047058823529411764 \
--cuda \
--low_vram_mode \
--mode lunet \  
--mixed_precision bf16 \
--max_train_steps 5 \
--max_f_train_steps 10 \
--max_adv_train_steps 30 \
--learning_rate 0.0001 \
--pgd_alpha 0.005 \
--lora_rank 4 \
--prior_loss_weight 0.1 \
--fused_weight 0.00001 \
--output_dir output/ \
--class_data_dir data/contrast/ \
--instance_data_dir data/img/ \
--target_image_path data/nips.png \

2024-01-18 04:35:48.110534: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-18 04:35:48.110636: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-18 04:35:48.217601: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-18 04:35:48.435003: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-18 04:35:50.585614: W tensorflow/comp

After seeing "==Saved misted image to ...", you are done with misting images. Check your outputs in the folder `outputs/`.